In [1]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [2]:
!pip install gdown

In [3]:
! rm -rf video.mp4

In [4]:
import gdown

# Update with your file's specific ID
file_id = "15C3ZIbvy4CemGBOmIkrGK7H7a2tpYaBJ"
url = f"https://drive.google.com/uc?id={file_id}"

output = "people-counting.mp4"
gdown.download(url, output, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=15C3ZIbvy4CemGBOmIkrGK7H7a2tpYaBJ
To: /content/people-counting.mp4
100%|██████████| 42.5M/42.5M [00:00<00:00, 43.1MB/s]


'people-counting.mp4'

In [1]:
import os
HOME = os.getcwd()
print(HOME)

/content


In [2]:
SOURCE_VIDEO_PATH = os.path.join(HOME, "people-counting.mp4")

In [3]:
# Pip install method (recommended)

!pip install "ultralytics<=8.3.40"

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics 8.3.40 🚀 Python-3.11.11 torch-2.6.0+cu124 CPU (Intel Xeon 2.20GHz)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 43.9/107.7 GB disk)


In [4]:
!pip install supervision==0.3.0

from IPython import display
display.clear_output()

import supervision
print("supervision.__version__:", supervision.__version__)

supervision.__version__: 0.3.0


In [5]:
# settings
MODEL = "yolov8x.pt"

In [6]:
from ultralytics import YOLO

model = YOLO(MODEL)
model.fuse()

YOLOv8x summary (fused): 268 layers, 68,200,608 parameters, 0 gradients, 257.8 GFLOPs


In [11]:
from ultralytics import YOLO

# Load YOLO model
model = YOLO('yolov8s.pt')

# Get class names from the model
class_names = model.names  # Dictionary {class_id: class_name}

# Print all class IDs and names
for class_id, class_name in class_names.items():
    print(f"Class ID: {class_id}, Class Name: {class_name}")

100%|██████████| 21.5M/21.5M [00:00<00:00, 145MB/s]


Class ID: 0, Class Name: person
Class ID: 1, Class Name: bicycle
Class ID: 2, Class Name: car
Class ID: 3, Class Name: motorcycle
Class ID: 4, Class Name: airplane
Class ID: 5, Class Name: bus
Class ID: 6, Class Name: train
Class ID: 7, Class Name: truck
Class ID: 8, Class Name: boat
Class ID: 9, Class Name: traffic light
Class ID: 10, Class Name: fire hydrant
Class ID: 11, Class Name: stop sign
Class ID: 12, Class Name: parking meter
Class ID: 13, Class Name: bench
Class ID: 14, Class Name: bird
Class ID: 15, Class Name: cat
Class ID: 16, Class Name: dog
Class ID: 17, Class Name: horse
Class ID: 18, Class Name: sheep
Class ID: 19, Class Name: cow
Class ID: 20, Class Name: elephant
Class ID: 21, Class Name: bear
Class ID: 22, Class Name: zebra
Class ID: 23, Class Name: giraffe
Class ID: 24, Class Name: backpack
Class ID: 25, Class Name: umbrella
Class ID: 26, Class Name: handbag
Class ID: 27, Class Name: tie
Class ID: 28, Class Name: suitcase
Class ID: 29, Class Name: frisbee
Class ID:

##### Install face detection pre trained models for age gender 

In [7]:
!pip install tensorflow==2.12.0 deepface==0.0.79

##### Install ffmpeg-python to get video metadata

In [8]:
!pip install ffmpeg-python

##### Face detection model and imports

In [9]:
!git clone https://github.com/chandrikadeb7/Face-Mask-Detection.git


fatal: destination path 'Face-Mask-Detection' already exists and is not an empty directory.


In [ ]:
from tensorflow.keras.utils import img_to_array
from tensorflow.keras.models import load_model

model_path = "Face-Mask-Detection/mask_detector.model"
mask_model = load_model(model_path)


### Tracking only when entering and exiting

In [13]:
import supervision as sv
from ultralytics import YOLO
import cv2
import numpy as np
import json
from datetime import datetime, timedelta
import os
from deepface import DeepFace
import ffmpeg
from datetime import timezone
import math
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

# ===== Configuration =====
TARGET_VIDEO_PATH = "output_video_final_trackingID_based.mp4"
JSON_OUTPUT_PATH = "detection_data_final_trackingID_based.json"

# Define polygonal areas for counting
area1 = np.array([(1169, 1678+50), (1942, 2025+50), (1816, 2102+50), (1085, 1703+50)], np.int32)
area2 = np.array([(1040, 1710+50), (1771, 2117+50), (1673, 2142+50), (981, 1713+50)], np.int32)

# Define restricted area (example coordinates - adjust as needed)
restricted_area = np.array([(500, 500), (800, 500), (800, 800), (500, 800)], np.int32)

# Object classes (COCO dataset)
BAG_CLASSES = [24, 26, 28]  # backpack, handbag, suitcase
CAT_CLASS = 15
DOG_CLASS = 16

# Mask detection labels
MASK_LABELS = ["mask", "no_mask", "improper"]

# ===== Initialize =====
total_count = 0
entering_count = 0
exiting_count = 0
restricted_area_count = 0  # New counter for restricted area
tracker_states = {}  # Tracks area crossings
detection_data = {}  # Stores entry/exit information
restricted_people = set()  # Track people who entered restricted area

# ===== Helper Functions =====
def extract_video_metadata(video_path):
    """Extract all available metadata from a video file using ffmpeg-python."""
    try:
        probe = ffmpeg.probe(video_path)
        metadata = {}

        # ===== 1. General Video Information =====
        if "format" in probe:
            format_info = probe["format"]
            metadata.update({
                "filename": format_info.get("filename"),
                "format_name": format_info.get("format_name"),
                "format_long_name": format_info.get("format_long_name"),
                "duration_seconds": float(format_info.get("duration", 0)),
                "size_bytes": int(format_info.get("size", 0)),
                "bitrate": int(format_info.get("bit_rate", 0)),
            })

            # Extract creation_time (if available)
            if "tags" in format_info:
                metadata.update({
                    "creation_time": format_info["tags"].get("creation_time"),
                    "encoder": format_info["tags"].get("encoder"),
                })

        # ===== 2. Video Stream Metadata =====
        video_streams = [s for s in probe["streams"] if s["codec_type"] == "video"]
        if video_streams:
            video_info = video_streams[0]
            metadata.update({
                "video_codec": video_info.get("codec_name"),
                "width": int(video_info.get("width", 0)),
                "height": int(video_info.get("height", 0)),
                "fps": eval(video_info.get("avg_frame_rate", "0/1")),  # e.g., "30/1" → 30.0
            })

            # Extract device-specific metadata (iPhone, Android, etc.)
            if "tags" in video_info:
                metadata.update({
                    "device_model": video_info["tags"].get("com.apple.quicktime.model"),
                    "software": video_info["tags"].get("software"),
                })

        # ===== 3. Audio Stream Metadata =====
        audio_streams = [s for s in probe["streams"] if s["codec_type"] == "audio"]
        if audio_streams:
            audio_info = audio_streams[0]
            metadata.update({
                "audio_codec": audio_info.get("codec_name"),
                "sample_rate": int(audio_info.get("sample_rate", 0)),
                "channels": int(audio_info.get("channels", 0)),
            })

        # ===== 4. GPS Coordinates (if recorded) =====
        if "format" in probe and "tags" in probe["format"]:
            tags = probe["format"]["tags"]
            if "location" in tags:  # Some Android devices store GPS here
                metadata["gps_coordinates"] = tags["location"]
            elif "com.apple.quicktime.location.ISO6709" in tags:  # iPhone GPS
                metadata["gps_coordinates"] = tags["com.apple.quicktime.location.ISO6709"]

        # ===== 5. Convert ISO Timestamp to Readable Format =====
        if "creation_time" in metadata:
            try:
                dt = datetime.strptime(metadata["creation_time"].split(".")[0], "%Y-%m-%dT%H:%M:%S")
                dt = dt.replace(tzinfo=timezone.utc)
                metadata["creation_time_utc"] = dt.strftime("%Y-%m-%d %H:%M:%S UTC")
                metadata["creation_time_local"] = dt.astimezone().strftime("%Y-%m-%d %H:%M:%S %Z")
                metadata["recording_time"] = metadata["creation_time_local"]  # For backward compatibility
            except Exception:
                pass

        return metadata

    except ffmpeg.Error as e:
        print(f"FFmpeg error: {e.stderr.decode('utf-8')}")
        return None
    except Exception as e:
        print(f"Error: {str(e)}")
        return None

def detect_mask(face_img):
    """Detect if person is wearing a mask"""
    try:
        face = cv2.cvtColor(face_img, cv2.COLOR_BGR2RGB)
        face = cv2.resize(face, (224, 224))
        face = img_to_array(face)
        face = preprocess_input(face)
        face = np.expand_dims(face, axis=0)

        preds = mask_model.predict(face)[0]
        max_idx = np.argmax(preds)
        return MASK_LABELS[max_idx], float(preds[max_idx])
    except Exception as e:
        print(f"Mask detection error: {str(e)}")
        return "unknown", 0.0

def is_carried(obj_bbox, person_bbox):
    """Check if an object is being carried by a person"""
    px1, py1, px2, py2 = person_bbox
    ox1, oy1, ox2, oy2 = obj_bbox
    obj_center_y = (oy1 + oy2) / 2
    lower_half_threshold = py1 + (py2 - py1) * 0.6
    overlap = (ox1 > px1) and (ox2 < px2) and (oy1 > py1) and (oy2 < py2)
    in_carry_position = obj_center_y > lower_half_threshold
    return overlap and in_carry_position

def analyze_person(frame, bbox, objects):
    """Analyze a person's attributes at entry/exit points"""
    x1, y1, x2, y2 = map(int, bbox)
    face_roi = frame[y1:y2, x1:x2]

    # Age/gender detection
    try:
        analysis = DeepFace.analyze(face_roi, actions=['age', 'gender'], enforce_detection=False)
        gender = analysis[0]['dominant_gender']
        age = analysis[0]['age']
    except:
        gender, age = "Unknown", "Unknown"

    # Mask detection
    mask_status, mask_conf = detect_mask(face_roi)

    # Check for carried items
    carried_items = []
    for obj_bbox, _, class_id in objects:
        if is_carried(obj_bbox, bbox):
            if class_id in BAG_CLASSES:
                carried_items.append("bag")
            elif class_id == CAT_CLASS:
                carried_items.append("cat")
            elif class_id == DOG_CLASS:
                carried_items.append("dog")

    return gender, age, mask_status, mask_conf, carried_items if carried_items else "no objects"

# ===== Main Processing =====
video_metadata = extract_video_metadata(SOURCE_VIDEO_PATH)
video_info = sv.VideoInfo.from_video_path(SOURCE_VIDEO_PATH)

# Get recording time from metadata or use current time as fallback
try:
    recording_time = datetime.strptime(video_metadata["creation_time"].split(".")[0], "%Y-%m-%dT%H:%M:%S")
    recording_time = recording_time.replace(tzinfo=timezone.utc)
except (KeyError, ValueError):
    print("Warning: Using current time as recording time fallback")
    recording_time = datetime.now(timezone.utc)

with sv.VideoSink(TARGET_VIDEO_PATH, video_info) as sink:
    for frame_number, result in enumerate(
        model.track(source=SOURCE_VIDEO_PATH, tracker="bytetrack.yaml", show=False, stream=True, persist=True)
    ):
        frame = result.orig_img
        detections = sv.Detections.from_yolov8(result)

        # Separate detections
        people = []
        objects = []
        if result.boxes.id is not None:
            tracker_ids = result.boxes.id.cpu().numpy().astype(int)
            for i, (bbox, conf, class_id) in enumerate(zip(detections.xyxy, detections.confidence, detections.class_id)):
                if class_id == 0:  # Person
                    tracker_id = tracker_ids[i] if i < len(tracker_ids) else None
                    people.append((bbox, conf, tracker_id))
                elif class_id in BAG_CLASSES + [CAT_CLASS, DOG_CLASS]:
                    objects.append((bbox, conf, class_id))

        # Process each person's area crossings
        for bbox, conf, tracker_id in people:
            x1, y1, x2, y2 = bbox
            bottom_center = (int((x1+x2)/2), int(y2))

            # Check area crossings
            in_area1 = cv2.pointPolygonTest(area1, bottom_center, False) >= 0
            in_area2 = cv2.pointPolygonTest(area2, bottom_center, False) >= 0
            in_restricted = cv2.pointPolygonTest(restricted_area, bottom_center, False) >= 0

            current_area = None
            if in_area1: current_area = "area1"
            elif in_area2: current_area = "area2"

            # Initialize tracker only when entering monitored areas
            if tracker_id not in tracker_states and current_area:
                tracker_states[tracker_id] = {
                    'current_area': current_area,
                    'last_area': None,
                    'entry_time': None,
                    'entered_restricted': False
                }
                detection_data[tracker_id] = {
                    "tracker_id": int(tracker_id),
                    "gender": "Unknown",
                    "age": "Unknown",
                    "carrying": "none",
                    "mask_status": "unknown",
                    "mask_confidence": 0.0,
                    "confidence": float(conf),
                    "entry_time": None,
                    "exit_time": None,
                    "entry_frame": None,
                    "exit_frame": None,
                    "entry_mask_status": None,
                    "exit_mask_status": None,
                    "entered_restricted": False,
                    "restricted_entry_time": None,
                    "restricted_exit_time": None
                }

            # Only process if person is in our tracking system (entered monitored area)
            if tracker_id in tracker_states:
                # Check restricted area entry
                if in_restricted and not tracker_states[tracker_id]['entered_restricted']:
                    tracker_states[tracker_id]['entered_restricted'] = True
                    detection_data[tracker_id]['entered_restricted'] = True
                    detection_data[tracker_id]['restricted_entry_time'] = (
                        recording_time + timedelta(seconds=frame_number / video_info.fps)
                    ).strftime("%Y-%m-%d %H:%M:%S")
                    restricted_area_count += 1
                    restricted_people.add(tracker_id)

                    # Draw alert for restricted area entry
                    cv2.putText(frame, "RESTRICTED AREA ENTRY!", (int(x1), int(y1)-30),
                               cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

                # Check restricted area exit
                elif not in_restricted and tracker_states[tracker_id]['entered_restricted']:
                    detection_data[tracker_id]['restricted_exit_time'] = (
                        recording_time + timedelta(seconds=frame_number / video_info.fps)
                    ).strftime("%Y-%m-%d %H:%M:%S")

                # Handle area transitions
                if current_area != tracker_states[tracker_id]['current_area']:
                    # Entry event (area2)
                    if current_area == "area1" and tracker_states[tracker_id]['last_area'] == "area2":
                        # Analyze person attributes
                        gender, age, mask_status, mask_conf, carrying = analyze_person(frame, bbox, objects)

                        # Update detection data
                        entry_time = recording_time + timedelta(seconds=frame_number / video_info.fps)
                        detection_data[tracker_id].update({
                            "tracker_id": int(tracker_id),
                            "gender": gender,
                            "age": age,
                            "carrying": carrying,
                            "mask_status": mask_status,
                            "mask_confidence": mask_conf,
                            "entry_time": entry_time.strftime("%Y-%m-%d %H:%M:%S"),
                            "entry_frame": frame_number,
                            "entry_mask_status": mask_status,
                            "entry_mask_confidence": mask_conf
                        })
                        entering_count += 1
                        total_count += 1

                    # Exit event (area1 after area2)
                    elif current_area == "area2" and tracker_states[tracker_id]['last_area'] == "area1":
                        # Analyze person attributes again at exit
                        gender, age, mask_status, mask_conf, carrying = analyze_person(frame, bbox, objects)

                        # Update detection data
                        exit_time = recording_time + timedelta(seconds=frame_number / video_info.fps)
                        detection_data[tracker_id].update({
                            "tracker_id": int(tracker_id),
                            "gender": gender,
                            "age": age,
                            "carrying": carrying,
                            "mask_status": mask_status,
                            "mask_confidence": mask_conf,
                            "exit_time": exit_time.strftime("%Y-%m-%d %H:%M:%S"),
                            "exit_frame": frame_number,
                            "exit_mask_status": mask_status,
                            "exit_mask_confidence": mask_conf
                        })
                        exiting_count += 1
                        total_count += 1

                    # Update tracker state
                    tracker_states[tracker_id]['last_area'] = tracker_states[tracker_id]['current_area']
                    tracker_states[tracker_id]['current_area'] = current_area

                # Draw bounding box if person is being tracked
                if detection_data[tracker_id]['entry_time']:
                    person = detection_data[tracker_id]
                    label = f"ID: {tracker_id} | {person['gender']}, {person['age']} | Mask: {person['mask_status']}"
                    if person['entered_restricted']:
                        # Highlight people who entered restricted area
                        cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 0, 255), 3)
                        label += " | RESTRICTED"
                    else:
                        cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
                    cv2.putText(frame, label, (int(x1), int(y1)-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 1)

        # Draw counters and areas
        cv2.polylines(frame, [area1], isClosed=True, color=(255, 0, 0), thickness=2)
        cv2.polylines(frame, [area2], isClosed=True, color=(0, 255, 0), thickness=2)
        cv2.polylines(frame, [restricted_area], isClosed=True, color=(0, 0, 255), thickness=2)
        cv2.putText(frame, "RESTRICTED AREA", (restricted_area[0][0], restricted_area[0][1]-10),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

        cv2.putText(frame, f"Total: {total_count}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
        cv2.putText(frame, f"Entering: {entering_count}", (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.putText(frame, f"Exiting: {exiting_count}", (50, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        cv2.putText(frame, f"Restricted Area: {restricted_area_count}", (50, 200),
                   cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        sink.write_frame(frame)

# ===== Save Results =====
json_output = {
    "video_metadata": video_metadata,
    "processing_time": datetime.now(timezone.utc).strftime("%Y-%m-%d %H:%M:%S %Z"),
    "summary": {
        "total_people": int(total_count),
        "total_entering": int(entering_count),
        "total_exiting": int(exiting_count),
        "restricted_area_entries": int(restricted_area_count),
        "restricted_people_ids": [int(id) for id in restricted_people],
        "fps": float(video_info.fps),
        "duration_seconds": float(video_info.total_frames / video_info.fps)
    },
    "detections": {
        int(tracker_id): data for tracker_id, data in detection_data.items()
    }
}

with open(JSON_OUTPUT_PATH, "w") as f:
    json.dump(json_output, f, indent=4)

print(f"Processing complete. Results saved to {TARGET_VIDEO_PATH} and {JSON_OUTPUT_PATH}")
print(f"Total: {total_count}, Entering: {entering_count}, Exiting: {exiting_count}")
print(f"Restricted area entries: {restricted_area_count}")


video 1/1 (frame 1/592) /content/people-counting.mp4: 384x640 1 potted plant, 5198.6ms
video 1/1 (frame 2/592) /content/people-counting.mp4: 384x640 1 car, 1 potted plant, 4178.9ms
video 1/1 (frame 3/592) /content/people-counting.mp4: 384x640 1 car, 1 potted plant, 4028.9ms
video 1/1 (frame 4/592) /content/people-counting.mp4: 384x640 1 car, 1 potted plant, 5365.4ms
video 1/1 (frame 5/592) /content/people-counting.mp4: 384x640 1 car, 1 potted plant, 3970.3ms
video 1/1 (frame 6/592) /content/people-counting.mp4: 384x640 1 person, 1 car, 1 potted plant, 4011.5ms
video 1/1 (frame 7/592) /content/people-counting.mp4: 384x640 1 person, 1 car, 1 potted plant, 5307.7ms
video 1/1 (frame 8/592) /content/people-counting.mp4: 384x640 1 person, 1 car, 1 potted plant, 4056.9ms
video 1/1 (frame 9/592) /content/people-counting.mp4: 384x640 1 person, 1 car, 1 potted plant, 3513.2ms
video 1/1 (frame 10/592) /content/people-counting.mp4: 384x640 1 person, 1 car, 1 bench, 1 potted plant, 2932.0ms
video 

Action: gender: 100%|██████████| 2/2 [00:01<00:00,  1.62it/s]

1/1 [==============================] - 0s 60ms/step


video 1/1 (frame 65/592) /content/people-counting.mp4: 384x640 3 persons, 1 car, 1 bench, 1 potted plant, 2331.2ms
video 1/1 (frame 66/592) /content/people-counting.mp4: 384x640 3 persons, 1 car, 1 bench, 1 potted plant, 3317.0ms
video 1/1 (frame 67/592) /content/people-counting.mp4: 384x640 3 persons, 1 car, 1 bench, 1 potted plant, 2392.3ms
video 1/1 (frame 68/592) /content/people-counting.mp4: 384x640 3 persons, 1 car, 1 bench, 1 potted plant, 2345.5ms
video 1/1 (frame 69/592) /content/people-counting.mp4: 384x640 3 persons, 1 car, 1 bench, 1 potted plant, 2363.6ms
video 1/1 (frame 70/592) /content/people-counting.mp4: 384x640 3 persons, 1 car, 1 bench, 1 potted plant, 2554.4ms
video 1/1 (frame 71/592) /content/people-counting.mp4: 384x640 3 persons, 1 car, 1 bench, 1 potted plant, 3046.7ms
video 1/1 (frame 72/592) /content/people-counting.mp4: 384x640 3 persons, 1 bench, 1 potted plant, 2373.5ms
video 1/1 (frame 73/592) /content/people-counting.mp4: 384x640 3 persons, 1 bench, 1 po

Action: gender: 100%|██████████| 2/2 [00:01<00:00,  1.68it/s]

...

1/1 [==============================] - 0s 52ms/step


video 1/1 (frame 424/592) /content/people-counting.mp4: 384x640 1 person, 1 potted plant, 2349.8ms
video 1/1 (frame 425/592) /content/people-counting.mp4: 384x640 1 person, 1 potted plant, 2376.7ms
video 1/1 (frame 426/592) /content/people-counting.mp4: 384x640 1 person, 1 potted plant, 2368.0ms
video 1/1 (frame 427/592) /content/people-counting.mp4: 384x640 1 person, 1 potted plant, 2513.9ms
video 1/1 (frame 428/592) /content/people-counting.mp4: 384x640 1 person, 1 potted plant, 3174.9ms
video 1/1 (frame 429/592) /content/people-counting.mp4: 384x640 1 person, 2 potted plants, 2365.0ms
video 1/1 (frame 430/592) /content/people-counting.mp4: 384x640 1 person, 2 potted plants, 2398.1ms
video 1/1 (frame 431/592) /content/people-counting.mp4: 384x640 1 person, 1 potted plant, 2341.7ms
video 1/1 (frame 432/592) /content/people-counting.mp4: 384x640 1 person, 1 potted plant, 2860.4ms
video 1/1 (frame 433/592) /content/people-counting.mp4: 384x640 1 person, 1 potted plant, 2759.0ms
video 1/

Action: gender: 100%|██████████| 2/2 [00:01<00:00,  1.69it/s]

1/1 [==============================] - 0s 53ms/step


video 1/1 (frame 539/592) /content/people-counting.mp4: 384x640 1 person, 1 potted plant, 2384.4ms
video 1/1 (frame 540/592) /content/people-counting.mp4: 384x640 1 person, 1 potted plant, 2533.2ms
video 1/1 (frame 541/592) /content/people-counting.mp4: 384x640 1 person, 3164.9ms
video 1/1 (frame 542/592) /content/people-counting.mp4: 384x640 1 person, 2371.7ms
video 1/1 (frame 543/592) /content/people-counting.mp4: 384x640 1 person, 2341.0ms
video 1/1 (frame 544/592) /content/people-counting.mp4: 384x640 1 person, 2367.9ms
video 1/1 (frame 545/592) /content/people-counting.mp4: 384x640 1 person, 2898.6ms
video 1/1 (frame 546/592) /content/people-counting.mp4: 384x640 1 person, 2727.6ms
video 1/1 (frame 547/592) /content/people-counting.mp4: 384x640 1 person, 2399.7ms
video 1/1 (frame 548/592) /content/people-counting.mp4: 384x640 1 person, 1 potted plant, 2370.6ms
video 1/1 (frame 549/592) /content/people-counting.mp4: 384x640 1 person, 1 potted plant, 2366.9ms
video 1/1 (frame 550/59